In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
# Чтение/переименование

def read_rename(filename):
    df = pd.read_csv(filename,index_col='Unnamed: 0')
    df.rename(columns={'SeriousDlqin2yrs':'Факт просрочки кредита', 'age':'Возраст заемщика', 
                         'RevolvingUtilizationOfUnsecuredLines':'Отношение баланса на кредитках к лимиту', 
                         'NumberOfTime30-59DaysPastDueNotWorse':'Просрочки 30-59', 'DebtRatio':'Отношение доходы-расходы', 
                         'MonthlyIncome':'Доход', 'NumberOfOpenCreditLinesAndLoans':'Число открытых кредитов', 
                         'NumberOfTimes90DaysLate':'Просрочки >90',
                         'NumberOfTime60-89DaysPastDueNotWorse':'Просрочки 60-89',
                         'NumberRealEstateLoansOrLines':'Число ипотек', 'NumberOfDependents':'Иждивенцы'},
               inplace=True)
    return df

In [3]:
df_train = read_rename('cs-training.csv')

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
Факт просрочки кредита                     150000 non-null int64
Отношение баланса на кредитках к лимиту    150000 non-null float64
Возраст заемщика                           150000 non-null int64
Просрочки 30-59                            150000 non-null int64
Отношение доходы-расходы                   150000 non-null float64
Доход                                      120269 non-null float64
Число открытых кредитов                    150000 non-null int64
Просрочки >90                              150000 non-null int64
Число ипотек                               150000 non-null int64
Просрочки 60-89                            150000 non-null int64
Иждивенцы                                  146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 13.7 MB


#### Создадим конвейер. 
Поскольку есть некоторые пропуски в данных, конвейер будет состоять из трех этапов:
<ol>
    <li>Импутации</li>
    <li>Масштабирования</li>
    <li>Алгоритма обучения</li>
</ol>

In [7]:
# Конвейер
conveyor = Pipeline([('imputation', SimpleImputer(strategy='median')),
                    ('scaling', StandardScaler()),
                    ('classifier', 
                     LogisticRegression(penalty='l2',C=1, random_state=0, max_iter=500)),
                    ])

# Кросс-валидация 
cv_avg = cross_val_score(conveyor, 
                         df_train.drop(columns='Факт просрочки кредита'), 
                         df_train['Факт просрочки кредита'],
                         scoring='roc_auc', 
                         cv=3).mean()

print('Средняя правильность на обучении по перекрестной проверке: {:.3f}'.format(cv_avg))

Средняя правильность на обучении по перекрестной проверке: 0.698
